In [1]:
%load_ext autoreload
%autoreload 1

%aimport pyscisci
import random

In [2]:
Npubs = 10
Nauthors = 3

database = pyscisci.BibDatabase()

for pid in range(10):
    pubdict = {'id':pid, 'year':2000+pid,
               'title':'', 
               'authors':random.sample(list(range(Nauthors)), 2),
               'references':list(range(pid+1)), 
               'citations':list(range(pid+1, Npubs))}
    database.add_pub(pyscisci.Publication(database).from_dict(pubdict))
    
for aid in range(Nauthors):
    authordict = {'id':aid, 'fullname':'Example Author{}'.format(aid),
                 'publications':[pid for pid in database.publication_ids if aid in database.get_pub(pid).authors]}
    database.add_author(pyscisci.Author(database).from_dict(authordict))
    
    
print(database.publication_years)
print(database.start_year, database.end_year)

print("{0} publications and {1} authors".format(database.n_publications, database.n_authors))

[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009]
2000 2009
10 publications and 3 authors


In [3]:
for pid in range(Npubs):
    pub = database.get_pub(pid)
    print(pub.id, pub.year, pub.ck(4), pub.c10, pub.total_citations, pub.disruption_index)
    print("references: {0} | citations: {1}".format(pub.references, pub.citations))
    print(pub.cocited_publications)
    print(pub.yearly_cnorm)
    print()

0 2000 4 9 9 -1.0
references: [0] | citations: [1, 2, 3, 4, 5, 6, 7, 8, 9]
{1, 2, 3, 4, 5, 6, 7, 8, 9}
defaultdict(<class 'float'>, {2001: 2.0, 2002: 1.5, 2003: 1.3333333333333333, 2004: 1.25, 2005: 1.2, 2006: 1.1666666666666667, 2007: 1.1428571428571428, 2008: 1.125, 2009: 1.1111111111111112})

1 2001 4 8 8 -0.8888888888888888
references: [0, 1] | citations: [2, 3, 4, 5, 6, 7, 8, 9]
{0, 2, 3, 4, 5, 6, 7, 8, 9}
defaultdict(<class 'float'>, {2002: 1.5, 2003: 1.3333333333333333, 2004: 1.25, 2005: 1.2, 2006: 1.1666666666666667, 2007: 1.1428571428571428, 2008: 1.125, 2009: 1.1111111111111112})

2 2002 4 7 7 -0.875
references: [0, 1, 2] | citations: [3, 4, 5, 6, 7, 8, 9]
{0, 1, 3, 4, 5, 6, 7, 8, 9}
defaultdict(<class 'float'>, {2003: 1.3333333333333333, 2004: 1.25, 2005: 1.2, 2006: 1.1666666666666667, 2007: 1.1428571428571428, 2008: 1.125, 2009: 1.1111111111111112})

3 2003 4 6 6 -0.8571428571428571
references: [0, 1, 2, 3] | citations: [4, 5, 6, 7, 8, 9]
{0, 1, 2, 4, 5, 6, 7, 8, 9}
default

In [4]:
for aid in range(Nauthors):
    author = database.get_author(aid)
    print(author.start_year, author.end_year)
    print(author.career_length, author.total_productivity, author.annual_productivity)
    print("Total citations: {0}, H-index: {1}".format(author.total_citations, author.hindex))
    print()
print(author.cummulative_yearly_citations)
print(author.author_timeline)

2000 2009
10 10 1.0
Total citations: 45, H-index: 5

2000 2009
10 7 0.7
Total citations: 32, H-index: 4

2001 2008
8 3 0.375
Total citations: 13, H-index: 2

{2001: 0.0, 2002: 1.0, 2003: 2.0, 2004: 3.0, 2005: 4.0, 2006: 6.0, 2007: 8.0, 2008: 10.0, 2009: 13.0}
[[2001, 8], [2005, 4], [2008, 1]]
